In [2]:
import pandas as pd 
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [3]:
students = pd.read_csv("students.csv")
password = 'Jesse156.'
db_name = 'students'

In [4]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password=password, database=db_name)

In [5]:
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

In [8]:
students.to_sql(name="students", con=connection, if_exists='replace', index=False)

30

In [7]:
query1 = "SELECT * FROM students"
studs = pd.read_sql(query1, con=connection)

In [154]:
courses = pd.read_csv("courses.csv")

In [155]:
courses.to_sql(name='courses', con=connection, if_exists='replace', index=False)
cors = pd.read_sql("SELECT * FROM courses", con=connection)

In [156]:
grades = pd.read_csv("grades.csv")

In [157]:
grades.to_sql(name='grades', con=connection, if_exists='replace', index=False)
grad = pd.read_sql("SELECT * FROM grades", con=connection)
grad

,grade_id,student_id,assignment_id,grade,course_id
0,0,0,4,85,1
1,1,0,5,70,1
2,2,0,6,90,1
3,3,1,7,70,2
4,4,1,8,80,2
...,...,...,...,...,...
102,102,28,9,79,2
103,103,28,10,82,2
104,104,29,26,80,7
105,105,29,27,100,7


In [158]:
assignments = pd.read_csv("assignments.csv")

In [159]:
assignments.to_sql(name='assignments', con=connection, if_exists='replace', index=False)
assign = pd.read_sql("SELECT * FROM assignments", con=connection)


In [160]:
full_cols = pd.read_sql("SELECT * FROM courses JOIN assignments ON assignments.course_id=courses.course_id JOIN grades ON assignments.assignment_id=grades.assignment_id JOIN students ON students.student_id=grades.student_id", con=connection)
full_cols.to_sql(name='student_info', con=connection, if_exists='replace', index=False)
# talk about full data drawbacks

107

In [161]:
query4 = pd.read_sql("SELECT student_id, SUM(assignments.weight * grades.grade) AS final_grade FROM assignments JOIN grades ON assignments.assignment_id = grades.assignment_id GROUP BY student_id", con=connection)
query4
#"CREATE VIEW final_grades AS (SELECT student_id, SUM(assignments.weight * grades.grade) AS result FROM assignments JOIN grades ON assignments.assignment_id = grades.assignment_id GROUP BY student_id)"
# SELECT s.student_id, s.name, s.gpa, s.major, s.year, s.gender, s.birthday, s.email, c.course_name, c.instructor, c.grade_avg, c.day, c.time, c.credit_hrs, fg.result AS final_grade
# FROM final_grades AS fg
# JOIN students AS s
# ON s.student_id = fg.student_id
# JOIN courses AS c
# ON s.course_id = c.cou

,student_id,final_grade
0,0,83.00
1,1,79.10
2,2,71.50
3,3,86.50
4,4,80.00
5,5,90.55
6,6,91.50
7,7,95.00
8,8,72.50
9,9,90.60


In [162]:
query5 = pd.read_sql("SELECT student_id, assignments.weight * grades.grade AS result FROM assignments JOIN grades ON assignments.assignment_id = grades.assignment_id" , con=connection)

In [163]:
stud_grade = pd.merge(students,query4[['student_id','final_grade']],on='student_id', how='inner')
grades_of_students= pd.merge(assign,grad[['assignment_id','student_id','grade']], on ='assignment_id', how='outer')
dic = grades_of_students.to_dict('records')
dics ={}
for i in dic: 
    if i['student_id'] not in dics: 
        dics[i['student_id']] = [] 
        dics[i['student_id']].append({i['assignment_name']:i['grade']})
    else: 
        dics[i['student_id']].append({i['assignment_name']:i['grade']})


In [164]:
full_df = pd.merge(stud_grade,courses[['course_id','course_name','instructor','grade_avg','day','time','credit_hrs']],on='course_id', how='inner')

In [165]:
full_df.to_sql(name="students_information", con=connection, if_exists='replace', index=False)

30

In [476]:
mongo_students = full_df.to_dict('records')
studes = studs.to_dict('records')
cours = cors.to_dict('records')
cors
for i in studes: 
    i['course_id'] = cours[i['course_id']]
lis = []
count = 0
for i in studes: 
    i['course_id']['cumulative_grades'] = dics[i['student_id']]
student_updated=studes
studes
dic = {}
for i in studes: 
    if i ['name'] not in dic: 
        dic[i['name']]=i['course_id']['cumulative_grades']

In [167]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import pprint

In [168]:
client = MongoClient('localhost', 27017)

In [169]:
db = client.college_students

In [170]:
student_collection = db.students

In [171]:
student_coll = db.studes

In [172]:
student_cols = db.studs_grades

In [173]:
student_coll.insert_many(studes)

In [174]:
student_collection.insert_many(mongo_students)

In [175]:
student_cols.insert_many(student_updated)

In [176]:
top_students= {'final_grade': {'$gt': 93,"$lt":97}}
top = student_collection.find(top_students)

In [177]:
# for e in top:
#     pprint.pprint(e)

In [178]:
bottom_students= {'grades': {'$gt': 0,"$lt":100}}
bottom = student_collection.find(bottom_students)

In [179]:
for i in bottom:
    pprint.pprint(e)

In [180]:
nest = full_df.to_json(orient="split")


In [479]:
query5 = pd.read_sql("SELECT name,major,year,course_name,final_grade,gpa,gender FROM students_information" , con=connection)

In [182]:
pip install py2neo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [212]:
from py2neo import Graph

In [484]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [485]:
type(graph)

py2neo.database.Graph

In [523]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [524]:
graph.run("CREATE (:GENDER {gender:'" + "female" + "'})")
graph.run("CREATE (:GENDER {gender:'" + "male" + "'})")
graph.run("CREATE (:GENDER {gender:'" + "nonbinary" + "'})")

(No data)

In [525]:
for i in range(len(cors)):
    row = cors.iloc[i]
    graph.run("CREATE (:COURSES {course_name:" + row['course_name'] + ",grade_avg:'" + row.grade_avg +  "',instructor:'" + row.instructor + "'})")

In [526]:
# for i in range(len(query5.year)):
#     row = query5.iloc[i]    
#     graph.run("MATCH (p:PERSON {name:'" + row['name'] +"'}), (s:COURSES {name:'" + row['course_name'] + "'}) CREATE (p)-[:TAKES_CLASS]->(s)")
# MATCH (n:Tag)
# WITH n.name AS name, COLLECT(n) AS nodelist, COUNT(*) AS count
# WHERE count > 1
# CALL apoc.refactor.mergeNodes(nodelist) YIELD node
# RETURN node
# graph.run("CREATE CONSTRAINT ON (_0:STUDENT) ASSERT _0.STUDENT IS UNIQUE;")

In [527]:
course_name_list = query5['course_name'].value_counts().index

In [528]:
for i in range(len(query5)):
    row = query5.iloc[i]
    graph.run("CREATE (:PERSON {name: '" + row['name'] + "', final_grade: " + str(row['final_grade']) + ", gpa: " + str(row['gpa']) + ", major: '" + row['major'] + "'})")

In [529]:
for i in range(3):
    row = query5.iloc[i]
    graph.run("CREATE (:PERSON {name: '" + row['name'] + "', final_grade: " + str(row['final_grade']) + ", gpa: " + str(row['gpa']) + ", major: '" + row['major'] + "', gender: '" + row['gender'] + "'})")

In [530]:
#new query 
for i in range(len(query5)):
    row = query5.iloc[i]
    graph.run(f""" MATCH
    (p:PERSON), 
    (c:COURSES)
    WHERE p.name = "{row['name']}" AND c.course_name ={row['course_name']} 
    CREATE (p)-[r:TAKES]->(c)""")
# MATCH 
# (p:PERSON), 
# (c:COURSE) WHERE p.name = "Ethan Patel" AND c.course_name = "Exploring Contemporary Art Movements" 
# CREATE (p)-[r:TAKES]->(c) RETURN p,c

In [531]:
for i in range(1,6):
    graph.run("CREATE (:YEAR {year:'" + str(i) + "'})")

In [532]:
for i in range(len(query5)):
    row = query5.iloc[i]
    graph.run(f""" MATCH
    (p:PERSON), 
    (y:YEAR)
    WHERE p.name = "{row['name']}" AND y.year ='{str(row['year'])}' 
    CREATE (p)-[r:IS_IN_YEAR]->(y)""")

In [533]:
for i in range(len(query5)):
    row = query5.iloc[i]
    graph.run(f""" MATCH
    (p:PERSON), 
    (g:GENDER)
    WHERE p.name = "{row['name']}" AND g.gender ='{row['gender']}' 
    CREATE (p)-[r:IDENTIFIES_AS]->(g)""")

In [283]:
# for i in range(len(query5)):
#     row = query5.iloc[i]
#     #First step: Create the person nodes, with the properties that won't be used to create relationships.
#     graph.run("CREATE (:PERSON {name:'" + row['name'] + "'})")
#     graph.run("CREATE (:COURSE {course:'" + row['course_name'] + "'})")
#     graph.run("CREATE (:YEAR {year:'" + str(row['year']) + "'})")
#     #What would creating relationships to companies and schools look like? Let's go to the cypher browser
#     graph.run("MATCH (p:PERSON {name:'" + row['name'] +"'}), (c:COURSES {name:'" + row['course_name'] + "'}) CREATE (p)-[:TAKES]->(c)")
    
#     #Same (similar) deal for schools
#     graph.run("MATCH (p:PERSON {name:'" + row['name'] +"'}), (y:YEAR {name:'" + str(row['year']) + "'}) CREATE (p)-[:IS_IN_YEAR]->(y)")
#     # graph.run("MATCH (p:PERSON {name:'" + row['name'] +"'}), (m:MAJOR {name:'" + row['major'] + "'}) CREATE (p)-[:MAJORS_IN]->(m)")